In [1]:
print("Jupyter Notebook is working!")


Jupyter Notebook is working!


In [2]:
!pip install pandas scikit-learn nltk


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import nltk
import string

nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\patil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df = pd.read_json("whatsapp_spam_collection_dataset.json")
df.head()


,message,label
0,You’ve been selected to receive a $1000 gift c...,spam
1,"Hi, did you get a chance to look at the report...",ham
2,Urgent! Your PayPal account has been locked. P...,spam
3,Let me know if you're free for a quick call to...,ham
4,Limited time offer! 60% off on all items at Be...,spam


In [5]:
df.head()


,message,label
0,You’ve been selected to receive a $1000 gift c...,spam
1,"Hi, did you get a chance to look at the report...",ham
2,Urgent! Your PayPal account has been locked. P...,spam
3,Let me know if you're free for a quick call to...,ham
4,Limited time offer! 60% off on all items at Be...,spam


In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   message  181 non-null    object
 1   label    180 non-null    object
dtypes: object(2)
memory usage: 3.0+ KB


In [7]:
df.isnull().sum()


message    0
label      1
dtype: int64

In [8]:
df['label'].value_counts()


label
spam    91
ham     89
Name: count, dtype: int64

In [9]:
df['message'] = df['message'].str.lower()


In [10]:
import re

df['message'] = df['message'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))


In [11]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')  # Download stopwords
stop_words = set(stopwords.words('english'))

df['message'] = df['message'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\patil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
df['label'] = df['label'].map({'spam': 1, 'ham': 0})


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)  # Limiting to 5000 features

# Transform the messages
X = vectorizer.fit_transform(df['message']).toarray()

# Store the labels (Spam = 1, Ham = 0)
y = df['label'].values

# Print the shape to check dimensions
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)


Shape of X: (181, 301)
Shape of y: (181,)


In [14]:
from sklearn.model_selection import train_test_split

# Splitting data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes to verify
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)


Training set shape: (144, 301) (144,)
Testing set shape: (37, 301) (37,)


## Training the Model using Naïve Bayes


In [16]:
from sklearn.naive_bayes import MultinomialNB

# Initialize the model
nb_classifier = MultinomialNB()

# Train (fit) the model using the training data
nb_classifier.fit(X_train, y_train)

# Check the model's accuracy on training data
train_accuracy = nb_classifier.score(X_train, y_train)
print("Training Accuracy:", train_accuracy)


ValueError: Input y contains NaN.

In [26]:
print(y_train)
print(pd.isnull(y_train).sum())  # Count NaN values


[ 1.  1.  0.  1.  0.  1.  0.  0.  1.  0.  1.  0.  1.  0.  1.  1.  1.  1.
  1.  1.  0.  0.  1.  0.  0.  1.  0.  1.  1.  1.  0.  1.  1.  0.  1.  0.
  0.  1.  1.  1.  1.  1.  1.  0.  0.  1.  1.  0.  0.  1.  0.  1.  1.  1.
  0.  0.  0.  1.  1.  1.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  1.  1.
  1.  0.  0.  0.  0.  0.  1.  1.  1.  1.  0.  0.  1.  0.  0.  0.  0.  0.
  1.  1.  0.  0.  1.  1.  1.  0.  1.  1.  0.  1.  1.  0.  0.  0.  1.  1.
  1.  0.  1.  1.  1.  1.  0.  1.  1.  0.  0.  0.  1.  0.  0.  0.  1.  0.
  1.  0.  1.  0.  1.  1.  0.  1.  0.  0. nan  1.  1.  0.  1.  0.  0.  0.]
1


In [27]:
from scipy.stats import mode

# Find the most common label (0 or 1)
most_common_label = mode(y_train, nan_policy="omit").mode[0]

# Fill NaN values in y_train
y_train = np.nan_to_num(y_train, nan=most_common_label)


IndexError: invalid index to scalar variable.

In [30]:
from scipy.stats import mode
import numpy as np

# Find the most common label (0 or 1)
most_common_label = mode(y_train, nan_policy="omit").mode

# Fill NaN values in y_train
y_train = np.nan_to_num(y_train, nan=most_common_label)


In [31]:
from sklearn.naive_bayes import MultinomialNB

# Initialize the Naive Bayes classifier
nb_classifier = MultinomialNB()

# Train (fit) the model using the training data
nb_classifier.fit(X_train, y_train)

# Check the model's accuracy on training data
train_accuracy = nb_classifier.score(X_train, y_train)
print("Training Accuracy:", train_accuracy)


Training Accuracy: 1.0


## Test the Model on Unseen Data

In [33]:
# Check the model's accuracy on test data
test_accuracy = nb_classifier.score(X_test, y_test)
print("Testing Accuracy:", test_accuracy)


Testing Accuracy: 1.0


In [34]:
# Predict labels for test data
y_pred = nb_classifier.predict(X_test)

# Print some predictions alongside actual labels
for i in range(10):  # Show first 10 examples
    print(f"Actual: {y_test[i]} - Predicted: {y_pred[i]}")


Actual: 0.0 - Predicted: 0.0
Actual: 1.0 - Predicted: 1.0
Actual: 0.0 - Predicted: 0.0
Actual: 0.0 - Predicted: 0.0
Actual: 0.0 - Predicted: 0.0
Actual: 0.0 - Predicted: 0.0
Actual: 1.0 - Predicted: 1.0
Actual: 0.0 - Predicted: 0.0
Actual: 1.0 - Predicted: 1.0
Actual: 0.0 - Predicted: 0.0


In [35]:
from sklearn.metrics import classification_report

# Generate report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        19
         1.0       1.00      1.00      1.00        18

    accuracy                           1.00        37
   macro avg       1.00      1.00      1.00        37
weighted avg       1.00      1.00      1.00        37



In [36]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

# Load dataset with encoding
df = pd.read_csv("spam.csv", encoding="latin1")

# Display first few rows
print(df.head())


     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


In [37]:
# Rename columns for clarity
df = df.rename(columns={"v1": "label", "v2": "message"})

# Drop unnecessary columns
df = df[["label", "message"]]

# Display the cleaned dataset
print(df.head())


  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [38]:
# Check for missing values
print(df.isnull().sum())

# Check for duplicate messages
print("Duplicate messages:", df.duplicated().sum())

# Check class distribution
print(df['label'].value_counts())


label      0
message    0
dtype: int64
Duplicate messages: 403
label
ham     4825
spam     747
Name: count, dtype: int64


In [39]:
# Remove duplicate messages
df = df.drop_duplicates(subset="message")

# Verify
print("Remaining messages after duplicate removal:", df.shape[0])


Remaining messages after duplicate removal: 5169


In [40]:
print(df.isnull().sum())


label      0
message    0
dtype: int64


In [41]:
df["label"] = df["label"].map({"ham": 0, "spam": 1})

# Verify
print(df.head())


   label                                            message
0      0  Go until jurong point, crazy.. Available only ...
1      0                      Ok lar... Joking wif u oni...
2      1  Free entry in 2 a wkly comp to win FA Cup fina...
3      0  U dun say so early hor... U c already then say...
4      0  Nah I don't think he goes to usf, he lives aro...


### text preprocessing - lowercase , removing special characters etc etc 

In [43]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Download stopwords if not already downloaded
nltk.download("stopwords")

stemmer = PorterStemmer()
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r"[^a-zA-Z]", " ", text)
    # Tokenize words
    words = text.split()
    # Remove stopwords and apply stemming
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    # Join words back into a string
    return " ".join(words)

# Apply preprocessing
df["cleaned_message"] = df["message"].apply(preprocess_text)

# Verify
print(df[["message", "cleaned_message"]].head())


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\patil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                             message  \
0  Go until jurong point, crazy.. Available only ...   
1                      Ok lar... Joking wif u oni...   
2  Free entry in 2 a wkly comp to win FA Cup fina...   
3  U dun say so early hor... U c already then say...   
4  Nah I don't think he goes to usf, he lives aro...   

                                     cleaned_message  
0  go jurong point crazi avail bugi n great world...  
1                              ok lar joke wif u oni  
2  free entri wkli comp win fa cup final tkt st m...  
3                u dun say earli hor u c alreadi say  
4               nah think goe usf live around though  


In [44]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_message'], df['label'], test_size=0.2, random_state=42)

In [45]:
# Convert text into TF-IDF features
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a new Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)

# Evaluate the model
y_pred = nb_classifier.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.97      1.00      0.98       889
           1       0.99      0.78      0.87       145

    accuracy                           0.97      1034
   macro avg       0.98      0.89      0.93      1034
weighted avg       0.97      0.97      0.97      1034



In [46]:
# Save the trained classifier and TF-IDF vectorizer
joblib.dump(nb_classifier, "spam_classifier.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

print("Updated model and vectorizer saved successfully!")


Updated model and vectorizer saved successfully!


In [47]:
# Load the updated model and vectorizer
loaded_classifier = joblib.load("spam_classifier.pkl")
loaded_vectorizer = joblib.load("tfidf_vectorizer.pkl")

# Example new message
new_message = ["Congratulations! You won a free iPhone! Claim now."]
new_message_transformed = loaded_vectorizer.transform(new_message)

# Predict
prediction = loaded_classifier.predict(new_message_transformed)

# Output result
print("Spam" if prediction[0] == 1 else "Ham")


Spam


In [48]:
# Load the updated model and vectorizer
loaded_classifier = joblib.load("spam_classifier.pkl")
loaded_vectorizer = joblib.load("tfidf_vectorizer.pkl")

# Example new message
new_message = ["hey what are you doing?"]
new_message_transformed = loaded_vectorizer.transform(new_message)

# Predict
prediction = loaded_classifier.predict(new_message_transformed)

# Output result
print("Spam" if prediction[0] == 1 else "Ham")

Ham


In [49]:
import sklearn
print(sklearn.__version__)


1.6.1


In [69]:
import sys
print(sys.executable)


C:\Users\patil\anaconda3\python.exe
